# Manoeuvring simulations
Many simulation model for ship manoeuvring have been developed in the field of ship hydrodynamics such as: the Abkowitz model <cite id="abkowitz_ship_1964">[NO_PRINTED_FORM]</cite> or the Norrbin model {cite}`norrbin_study_1960`.
This chapter will develop a general simulation model for ship manoeuvring, that can be further specified to become either the Abkowitz or Norbin model. Expressing the models on a general form is important in this research where many different models will be tested and compared.

In [ ]:
import matplotlib 
print(matplotlib.get_configdir())

In [ ]:
# %load imports.py
## Local packages:

%matplotlib inline
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = False  ## (To fix autocomplete)

## External packages:
import pandas as pd
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.set_option("display.max_columns", None)

import numpy as np
import os

import plotly.express as px 
import plotly.graph_objects as go

import seaborn as sns
import sympy as sp
from sympy.physics.mechanics import (dynamicsymbols, ReferenceFrame,
                                      Particle, Point)
from sympy.physics.vector.printing import vpprint, vlatex
from IPython.display import display, Math, Latex, Markdown
from src.substitute_dynamic_symbols import run, lambdify

import pyro

import sklearn
import pykalman
from statsmodels.sandbox.regression.predstd import wls_prediction_std
import statsmodels.api as sm

from scipy.integrate import solve_ivp

## Local packages:
from src.data import mdl

from src.symbols import *

import src.symbols as symbols
from src import prime_system
from src.models import regression
from src.visualization.regression import show_pred
from src.visualization.plot import track_plot
from myst_nb import glue

## Load models:
# (Uncomment these for faster loading):
import src.models.vmm_nonlinear_EOM  as vmm 

import matplotlib.pyplot as plt
if os.name == 'nt':
    plt.style.use('book.mplstyle')  # Windows
    
## Examples
from example_1 import ship_parameters, df_parameters

## Ship parameters
The Ocean Bird Wind Powered Car Carrier will be used as the reference ship of this book:
![](https://www.walleniusmarine.com/wp-content/uploads/2020/09/oceanbird-960x540.jpg)

In [ ]:
ps = prime_system.PrimeSystem(**ship_parameters)  # model

A model scale version of this ship has the following main dimensions:

In [ ]:
ship_parameters

## 3 DOF
A simulation model with only three degrees of fredom (DOF) : *surge*, *sway* and *yaw* is often sufficient to describe the ship's manoeuvring performance. The equation of motion can be expressed as <cite id="2k4ro">(Triantafyllou &#38; Hover, 2003)</cite>:

In [ ]:
display(Math(vlatex(vmm.X_eom)))
display(Math(vlatex(vmm.Y_eom)))
display(Math(vlatex(vmm.N_eom)))


Where $X_{force}$, $Y_{force}$ and $N_{force}$ are the hydrodynamic forces from the ship. So these equations holds for most of the different models for manoeuvring, except linear models where the nonlinear $r\cdot u$ term have been linearized. The difference in the methods is rather in how these hydrodynamic forces are calculated.

The hydrodynamic forces can be split into added masses (that depend on the accelerations: $\dot{u}$, $\dot{v}$, $\dot{r}$) and dampings (that depend on the velocities: $u$, $v$, $r$):

In [ ]:
display(Math(vlatex(vmm.fx_eq)))
display(Math(vlatex(vmm.fy_eq)))
display(Math(vlatex(vmm.mz_eq)))

$X_{\dot{u}}$, $X_{\dot{v}}$, $X_{\dot{r}}$ are the added masses: when the ship is accelerating in water, not only the mass of the ship is accelerating, but also some mass in the water needs to be accelerated. This is referred to as added masses. Sometimes added  masses such as $X_{\dot{v}}$ is neglected, as in the case above.

The specialization of the various simulation models can now be further categorized to the functions to calculate the damping forces: $X_{qs}$, $Y_{qs}$, $N_{qs}$. The subscript "qs" stands for quasi static.

In [ ]:
ship_parameters_prime = ps.prime(ship_parameters)

The general equation for all of the simulation models in this research can be written as: 

In [ ]:
display(Math(vlatex(vmm.X_eq)))
display(Math(vlatex(vmm.Y_eq)))
display(Math(vlatex(vmm.N_eq)))

The classic simulation methods express the damping forces as a truncated taylor series. Here is a rather simple model, that has been created by the author to be used in the followin examples.

In [ ]:
display(Math(vlatex(vmm.X_qs_eq)))
display(Math(vlatex(vmm.Y_qs_eq)))
display(Math(vlatex(vmm.N_qs_eq)))

## Similitude

### Prime system
The prime system uses ship length $L$, water density $\rho$ and total velocity $U^2=u^2+v^2$ to express physical quantities in nondimensional form. The quantities have a prime symbol (') attached to them when the prime system is used. The quantities are made nondimensional with the prime system by dividing with a denominator according to the table below:

In [ ]:
prime_system.df_prime.loc[['denominator']].transpose()

The manoeuvring models are often expressed in the prime system so that a nondimensional force $F'$ for instance can be expressed as a function of some coefficients $C$ and nondimensional velocity $v$:

In [ ]:
F_,v_, C_=sp.symbols("F' v' C")
F=sp.symbols("F")

eq = sp.Eq(F_, 
      C_*v_)
eq

This may look as a model where the force depend on the transverse velocity $v$ only, but this is actually not true. If this equation is converted converted back to SI units it is instead written:

In [ ]:
eq_F_prime = sp.Eq(F,prime_system.df_prime.loc['denominator','force']*F_)
eq_v_prime = sp.Eq(v,prime_system.df_prime.loc['denominator','linear_velocity']*v_)

eqs = [eq, eq_F_prime, eq_v_prime]

sp.Eq(F,sp.solve(eqs,F,F_,v_,dict=True)[0][F])

So it is now clear that the example force model above, also depends on the total velocity $U$. 

## Brix parameters
The hydrodynamic derivatives can be estimated with semi empirical formulas according to <cite id="ycziw">(Brix, 1993)</cite>.

In [ ]:
mask = df_parameters['prime'].notnull()
index = df_parameters.loc[mask,'prime'].index
coefficients=vmm.simulator.get_all_coefficients(sympy_symbols=False)
missing_coefficients = set(coefficients) - set(index)
missing_coefficients

In [ ]:
mask = df_parameters['prime'].notnull()
for index, parameter in df_parameters.loc[mask].iterrows():
    display(sp.Eq(parameter['symbol'],parameter['prime']))

In [ ]:
U_ = 2
glue("U_",U_);

In [ ]:
parameters=df_parameters['prime'].copy()

N_=30
df_captive_template = pd.DataFrame(index=np.arange(N_))
df_captive_template['U'] = U_
df_captive_template['u'] = df_captive_template['U']
df_captive_template['v'] = 0
df_captive_template['r'] = 0
df_captive_template['delta'] = 0

df_r = df_captive_template.copy()
df_r['r'] = np.linspace(-0.3,0,N_)
df_r['vary'] = 'r'

df_v = df_captive_template.copy()
df_v['v'] = np.linspace(0,0.5,N_)
df_v['u'] = np.sqrt(df_v['U']**2 - df_v['v']**2)
df_v['vary'] = 'v'

df_delta = df_captive_template.copy()
df_delta['delta'] = np.linspace(0.,0.4,N_)
df_delta['vary'] = 'delta'

df_captive = pd.concat([df_delta,df_v,df_r])

df_captive['X'] = run(vmm.simulator.X_qs_lambda, inputs=parameters, **df_captive)
df_captive['Y'] = run(vmm.simulator.Y_qs_lambda, inputs=parameters, **df_captive)
df_captive['N'] = run(vmm.simulator.N_qs_lambda, inputs=parameters, **df_captive)
df_captive = ps.prime(df_captive, U=df_captive['U'])  # Convert to prime system

ncols = len(df_captive['vary'].unique())
fig,axes=plt.subplots(ncols=ncols, nrows=3)

vary_labels={
    'delta' : r'\delta',
}
for col,(vary, df_) in enumerate(df_captive.groupby(by='vary')):
    
    for row,dof in enumerate(['X','Y','N']):
        ax = axes[row,col]
        df_.plot(x=vary, y=dof, ax=ax)
        ax.get_legend().set_visible(False)
        ax.set_xlabel('')
        ax.grid(True)
        axes[row,0].set_ylabel(f"${dof}'$")
        
        ylims = [df_captive[dof].min(),df_captive[dof].max()]
        if ylims[0]!=ylims[1]:
            ax.set_ylim(ylims)
        
    axes[-1,col].set_xlabel(f"${vary_labels.get(vary,vary)}'$")

The figure above shows the quasi static forces for the present ship with the hydrodynamic derivatives during a variation of rudder angle $\delta$,
         yaw rate $r$ and tranverse velocity $v$ and total velocity $U$={glue:}`U_` m/s

## Simulation

### Decoupling
There is a coupling between the sway and yaw equation thruogh the added masses. These equations need to be decoupled to be usable in a simulation model. This is done by first expressing the equation in matrix form:

In [ ]:
A = vmm.simulator.A
b = vmm.simulator.b
acceleration = sp.matrices.MutableDenseMatrix([u.diff(),v.diff(),r.diff()])
eq_simulator = sp.Eq(sp.UnevaluatedExpr(A)*sp.UnevaluatedExpr(acceleration),sp.UnevaluatedExpr(b))
Math(vlatex(eq_simulator))

The decoupled equations are obtained by inverting the intertia matrix:

This equation can be written in a cleaner way if the $S$ term is introduced:

In [ ]:
A_inv = A.inv()
S = sp.symbols('S')
eq_S=sp.Eq(S,-sp.fraction(A_inv[1,1])[1])

A_inv_S = A_inv.subs(eq_S.rhs,S)
eq_acceleration_matrix_clean = sp.Eq(sp.UnevaluatedExpr(acceleration),sp.UnevaluatedExpr(A_inv_S)*sp.UnevaluatedExpr(b))
Math(vlatex(eq_acceleration_matrix_clean))

Where $S$ is a helper variable defined as:

In [ ]:
eq_S

## Simulate data

In [ ]:
parameters=df_parameters['prime'].copy()

t_ = np.linspace(0,50,1000)
df = pd.DataFrame(index=t_)

df['u'] = 2
df['v'] = 0
df['r'] = 0
df['x0'] = 0
df['y0'] = 0
df['psi'] = 0
df['U'] = np.sqrt(df['u']**2 + df['v']**2)
df['beta'] = -np.arctan2(df['v'],df['u'])

df['delta'] = np.deg2rad(20)

result = vmm.simulator.simulate(df_=df, parameters=parameters, ship_parameters=ship_parameters, 
                                  control_keys=['delta'], primed_parameters=True,prime_system=ps)

result.save(path='first_simulation.csv')

In [ ]:
result.track_plot();

In [ ]:
fig = result.plot()
plt.tight_layout()

### Simulate parameter contributions

In [ ]:
df_result_prime = ps.prime(result.result, U=result.result['U'])

In [ ]:
X_ = sp.symbols('X_')
diff_eq_X = regression.DiffEqToMatrix(ode=vmm.X_qs_eq.subs(X_qs,X_), 
                                      label=X_, base_features=[delta,u,v,r])
X = diff_eq_X.calculate_features(data=df_result_prime)
X_parameters = df_parameters.groupby(by='dof').get_group('X')['prime'].dropna()
X_forces = X*X_parameters
X_forces.index = df_result_prime.index

Y_ = sp.symbols('Y_')
diff_eq_Y = regression.DiffEqToMatrix(ode=vmm.Y_qs_eq.subs(Y_qs,Y_), 
                                      label=Y_, base_features=[delta,u,v,r])
X = diff_eq_Y.calculate_features(data=df_result_prime)
Y_parameters = df_parameters.groupby(by='dof').get_group('Y')['prime'].dropna()
Y_forces = X*Y_parameters
Y_forces.index = df_result_prime.index

N_ = sp.symbols('N_')
diff_eq_N = regression.DiffEqToMatrix(ode=vmm.N_qs_eq.subs(N_qs,N_), 
                                      label=N_, base_features=[delta,u,v,r])
X = diff_eq_N.calculate_features(data=df_result_prime)
N_parameters = df_parameters.groupby(by='dof').get_group('N')['prime'].dropna()
N_forces = X*N_parameters
N_forces.index = df_result_prime.index

Here is an interavtive graph showing how the various hydrodynamic derivatives contribute to the forces:

In [ ]:
display(px.line(X_forces, y=X_forces.columns, width=800, height=350, title='X'))
display(px.line(Y_forces, y=Y_forces.columns, width=800, height=350, title='Y'))
display(px.line(N_forces, y=N_forces.columns, width=800, height=350, title='N'))